# get normalized output and denormalize it according to the models in models folder

In [50]:
import math
import locale
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from joblib import dump, load
import pickle
STATION = "Parco S. Chiara"
pollu = ["Biossido di Azoto", "Ossido di Carbonio", "PM10", "Ozono", "PM2.5", "Biossido Zolfo"]
POLLUTANT = "O3"

In [51]:
pollu_dict = {
    "AO2": "Biossido di Azoto",
    "CO2": "Ossido di Carbonio",
    "PM10": "PM10",
    "O3": "Ozono",
    "PM2.5": "PM2.5",
    "SO2": "Biossido Zolfo"
}
MODEL_FILE = f"models/{pollu_dict[POLLUTANT]}.joblib"
FILE = "data/test_day_data.csv"

In [52]:
try:
    model = load(MODEL_FILE)
    print("opened .joblib")
except KeyError:
    model = pickle.load(open(MODEL_FILE.split('.')[0]+'.sav', 'rb'))
    print("opened .sav")
day_data = pd.read_csv(FILE)

In [53]:
day_data.Data = pd.to_datetime(day_data.Data, format = "%Y-%m-%d %H:%M:%S")

In [64]:
day_data["real_ppm"]= day_data.Valore * model.predict(day_data.Data.dt.day_of_year[:,np.newaxis])

C:\Users\samue\AppData\Local\Temp\ipykernel_1724\2438649910.py:1: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  day_data["real_ppm"]= day_data.Valore * model.predict(day_data.Data.dt.day_of_year[:,np.newaxis])


test = pd.read_csv("data/APPA_data_merged.csv")
test.Data = pd.to_datetime(test.Data, format = "%Y-%m-%d %H:%M:%S")
test = test[test.Stazione == STATION]
test = test[test.Inquinante == pollu_dict[POLLUTANT]]
test[test.Data.dt.date == test.at[test.index[200],'Data'].date()].to_csv("data/test_day_data.csv")

In [65]:
day_data.to_csv(FILE)